https://bokeh.pydata.org/en/latest/docs/reference/palettes.html

In [30]:
from bokeh.io import (show, output_file)
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper
)
from bokeh.palettes import OrRd9 as palette
palette.reverse()
from bokeh.plotting import figure
#bokeh.sampledata.download()
from bokeh.sampledata.us_counties import data as counties
from bokeh.sampledata.unemployment import data as unemployment

import pandas as pd
import numpy as np

## Try using US zip code Mapping with Bokeh

In [31]:
df_zipcode_geo = pd.read_csv("US_ZIP_codes.csv")

In [32]:
df_counties = df_zipcode_geo[(df_zipcode_geo['STATE']=="CA") & ([len(str(item))>4 for item in df_zipcode_geo['ZIP']])]
df_counties_LA = df_counties[df_counties['PONAME']=='LOS ANGELES']

In [33]:
def getArrayFromGeokml(geokml):
    try:
        coord_string = geokml.strip("<Polygon><outerBoundaryIs><LinearRing><coordinates>").strip("</coordinates></LinearRing></outerBoundaryIs></Polygon>")
        coord_string.strip("<MultiGeometry>").strip("</MultiGeometry>")
        #print(coord_string)
        zip_array = np.array([[np.float(item.split(",")[0]),np.float(item.split(",")[1])] for item in coord_string.split()])
        return zip_array.T[0],zip_array.T[1]
    except:
        pass

In [34]:
df_2016_LA = pd.read_csv("2016_LA_Crimes.csv")

In [35]:
df_2016_LA_ds = df_2016_LA[["Zip code","Crime Code Description","DR Number"]]

In [36]:
df_zipbycrime = pd.DataFrame(df_2016_LA_ds.groupby(["Zip code","Crime Code Description"]).count()['DR Number'])

In [38]:
df_data = df_counties_LA.set_index("ZIP").join(df_zipbycrime.unstack()['DR Number']["ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT"])[['geometry',"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT"]].dropna().reset_index()

#df_data = df_counties_LA.set_index("ZIP").join(pd.DataFrame(df_zipbycrime.unstack()['DR Number']["ARSON"]))[['geometry',"ARSON"]].dropna()
zip_xs = [getArrayFromGeokml(zipCodeGeo)[0] for zipCodeGeo in df_data['geometry'].dropna()]
ZipNames = [str(zipname) for zipname in df_data[['geometry','ZIP']].dropna()['ZIP']]
zip_ys = [getArrayFromGeokml(zipCodeGeo)[1] for zipCodeGeo in df_data['geometry'].dropna()]
rates = list(df_data["ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT"])

color_mapper = LogColorMapper(palette=palette)

source = ColumnDataSource(data=dict(
    x=zip_xs,
    y=zip_ys,
    name=ZipNames,
    rate=rates,
))

TOOLS = "pan,wheel_zoom,reset,hover,save"

p = figure(
    title="ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT, 2010", tools=TOOLS,
    x_axis_location=None, y_axis_location=None
)

p.grid.grid_line_color = None


p.patches('x', 'y', source=source,
          fill_color={'field': 'rate', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.5)

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("Name", "@name"),
    ("Assault counts", "@rate"),
]

output_file("Assault.html")
show(p)